<a href="https://colab.research.google.com/github/Pedro-Boechat/trainee/blob/main/periodo4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
import time

# Binário

In [31]:
df = pd.read_csv('/content/creditcard.csv')

In [ ]:
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [ ]:
df.isna().sum()

In [33]:
df = df.dropna()

In [34]:
df['Class'].value_counts()

0.0    81100
1.0      198
Name: Class, dtype: int64

In [36]:
df = df.drop(columns='Time')

In [37]:
colunas = df.columns

## Normalizar

In [73]:
for i in colunas:
  df[i] = (df[i] - df[i].min()) / (df[i].max() - df[i].min())

In [ ]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.762583,0.660414,0.909721,0.565623,0.798543,0.159124,0.131668,0.765803,0.363363,0.236216,0.308548,0.437774,0.364897,0.732471,0.701250,0.405258,0.597465,0.595135,0.604822,0.615614,0.408828,0.645188,0.481783,0.660030,0.531926,0.224086,0.595303,0.394042,0.019400,0.0
1,0.961527,0.679003,0.765280,0.478470,0.808561,0.137239,0.123551,0.764960,0.298603,0.219554,0.660360,0.760630,0.590167,0.752040,0.586455,0.527165,0.563077,0.563349,0.524169,0.599172,0.390749,0.451615,0.507877,0.539604,0.543972,0.291007,0.579790,0.399230,0.000349,0.0
2,0.762696,0.590904,0.863209,0.472062,0.794397,0.212882,0.145736,0.775036,0.166905,0.243769,0.499724,0.568963,0.624889,0.749448,0.822259,0.089544,0.693562,0.572772,0.213737,0.629648,0.432034,0.749491,0.607457,0.436154,0.389684,0.234712,0.574744,0.388431,0.049097,0.0
3,0.793265,0.654247,0.864414,0.355570,0.806792,0.190653,0.131617,0.783077,0.180253,0.226795,0.361395,0.590476,0.593006,0.735188,0.411774,0.328374,0.502426,0.889970,0.364728,0.592043,0.400986,0.587621,0.471944,0.292186,0.693705,0.217115,0.587594,0.406007,0.016013,0.0
4,0.778299,0.712545,0.849526,0.474241,0.796811,0.144401,0.140675,0.742920,0.410841,0.279026,0.264458,0.659528,0.720530,0.637955,0.522974,0.407730,0.550055,0.585411,0.663428,0.623674,0.409601,0.755109,0.478458,0.682038,0.427610,0.370969,0.604648,0.428293,0.009075,0.0


## Verificar balanceamento

In [35]:
from sklearn.utils import resample
# Separar a maioria e minoria
df_majority = df[df['Class']==0]
df_minority = df[df['Class']==1]
# Downsample
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    
                                 n_samples=198) # Colocar aqui o downsample que quiser
# Combinar a maioria downsampled com a minoria
df = pd.concat([df_majority_downsampled, df_minority])

## Meu código

In [38]:
from sklearn.model_selection import train_test_split

X = df[colunas.drop('Class')]
Y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)


In [90]:
X_train.shape

(787, 29)

In [47]:
X_test = np.array(X_test)
X_train = np.array(X_train)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [40]:
n_features, n_samples = X_train.shape
weights1 = np.ones(n_samples)
weights2 = np.ones(n_samples)
hneurons = np.zeros(10)
bias1 = np.zeros(n_samples)
bias2 = np.zeros(10)


In [59]:
def activation1(x):
  return x * (x > 0)

def dact1(x):
    return 1. * (x > 0)

def activation2(x):
  resultado = 1/(1 + np.exp(-x))
  if resultado >= 0.5:
    return 1
  else:
    return 0

def dact2(x):
  return activation2(x) * (1.0 - activation2(x))


In [ ]:
print(X_train)

In [60]:
lista = []
predictions = []
for i in range(len(X_train)):
  lista = []
  for j in range(len(X_train[i])):
    forward = (X_train[i][j] * weights1[j]) + bias1[j]
    lista.append(forward)
  for k in range(len(hneurons)):
    z = np.sum(lista)
    ativada = activation1(z)
    hneurons[k] = ativada
  soma = []
  for j in range(len(hneurons)):
    forward = hneurons[j] * weights2[j] + bias2[j]
    soma.append(forward)
    z = np.sum(soma)
    ativada_output = activation2(z)
  
  predictions.append(ativada_output)

cost = []
for i in range(len(predictions)):
  custo = np.square(predictions[i] - y_train[i])
  cost.append(custo)
  custo_total = np.sum(cost)



In [74]:
derw2 = []
for i in range(len(hneurons)):

  for j in range(len(y_train)):

    dw2 = 2 * (predictions[j] - y_train[j]) * dact2(ativada_output) * np.sum(weights1)
    derw2.append(dw2)

In [65]:
1/(1 + np.exp(-z))

1.0

In [73]:
weights2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [71]:
print(len(derw2))

3160


In [72]:
print(sum(derw2))

0.0


## Daqui pra baixo são os códigos copiados ruins

In [ ]:
weights1 = []
weights2 = []
bias1 = 0
bias2 = 0
hidden = np.zeros(10)

def forward():
  for _ in range(iter):
    for idx, x_i in enumerate(X_train):
      forward = np.matmul(X_train,weights1) + bias1
      forward = activation1(forward)
      forward = np.matmul(forward, weights2) + bias2
      pred = activation2(forward)

      predictions = np.append(predictions, pred)
    
def backward():
  for i in range


In [75]:
import numpy as np


class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights1 = None
        self.weights2 = None
        self.bias1 = None
        self.bias2 = None
        self.hidden = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # init parameters
        self.weights1 = np.zeros(n_features)
        self.weights2 = np.zeros(10)
        self.bias1 = 0
        self.bias2 = 0
        self.hidden = np.zeros(10)

        y_ = np.array([1 if i > 0 else 0 for i in y])

        for _ in range(self.n_iters):

            for i in range(len(self.hidden)):

                for idx, x_i in enumerate(X):
                  
                  output = np.dot(x_i, self.weights1) + self.bias1
                  hneuron = self.activation_func(output)
                  self.hidden[i] = hneuron

            for idx, x_ii in enumerate(hidden):
                  output = np.dot(x_ii, self.weights2) + self.bias2
                  y_predicted = self.activation_func(output)

                



                # Perceptron update rule
            ERROR_output = y_predicted - y_train[idx]
            DELTA_output = ((-1)*(ERROR_output) * self.deriv_func(self.y_predicted))
            update1 = self.lr * (y_[idx] - y_predicted)

            self.weights += update * x_i
            self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x*(x > 0))

    def deriv_func(self, x):
        return np.where(1 * (x > 0))


# Testing
if __name__ == "__main__":
    # Imports
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn import datasets

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

    X = np.array(df[colunas.drop('Class')])
    Y = np.array(df['Class'])
    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, test_size=0.2, random_state=123
    )

    p = Perceptron(learning_rate=0.01, n_iters=1000)
    p.fit(X_train, y_train)
    predictions = p.predict(X_test)

    print("Perceptron classification accuracy", accuracy(y_test, predictions))

ValueError: ignored

Primeiro teste: Perceptron classification accuracy 0.9985955549313578

0.9982

array([[56831,    64],
       [   16,    51]])

0.76119

In [52]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

confusion_matrix(predictions, y_test)

array([[56831,    64],
       [   16,    51]])

In [64]:
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
import random

class MultiLayerPerceptron(BaseEstimator, ClassifierMixin): 
    def __init__(self, params=None):     
        if (params == None):
            self.inputLayer = 4                        # Input Layer
            self.hiddenLayer = 5                       # Hidden Layer
            self.outputLayer = 3                       # Outpuy Layer
            self.learningRate = 0.005                  # Learning rate
            self.max_epochs = 600                      # Epochs
            self.iasHiddenValue = -1                   # Bias HiddenLayer
            self.BiasOutputValue = -1                  # Bias OutputLayer
            self.activation = self.ativacao['sigmoid'] # Activation function
            self.deriv = self.derivada['sigmoid']
        else:
            self.inputLayer = params['InputLayer']
            self.hiddenLayer = params['HiddenLayer']
            self.OutputLayer = params['OutputLayer']
            self.learningRate = params['LearningRate']
            self.max_epochs = params['Epocas']
            self.BiasHiddenValue = params['BiasHiddenValue']
            self.BiasOutputValue = params['BiasOutputValue']
            self.activation = self.ativacao[params['ActivationFunction']]
            self.deriv = self.derivada[params['ActivationFunction']]
        
        'Starting Bias and Weights'
        self.WEIGHT_hidden = self.starting_weights(self.hiddenLayer, self.inputLayer)
        self.WEIGHT_output = self.starting_weights(self.OutputLayer, self.hiddenLayer)
        self.BIAS_hidden = np.array([self.BiasHiddenValue for i in range(self.hiddenLayer)])
        self.BIAS_output = np.array([self.BiasOutputValue for i in range(self.OutputLayer)])
        self.classes_number = 3 
        
    pass
    
    def starting_weights(self, x, y):
        return [[2  * random.random() - 1 for i in range(x)] for j in range(y)]

    ativacao = {
         'sigmoid': (lambda x: 1/(1 + np.exp(-x))),
            'tanh': (lambda x: np.tanh(x)),
            'Relu': (lambda x: x*(x > 0)),
               }
    derivada = {
         'sigmoid': (lambda x: x*(1-x)),
            'tanh': (lambda x: 1-x**2),
            'Relu': (lambda x: 1 * (x>0))
               }
 
    def Backpropagation_Algorithm(self, x):
        DELTA_output = []
        'Stage 1 - Error: OutputLayer'
        ERROR_output = self.output - self.OUTPUT_L2
        DELTA_output = ((-1)*(ERROR_output) * self.deriv(self.OUTPUT_L2))
        
        arrayStore = []
        'Stage 2 - Update weights OutputLayer and HiddenLayer'
        for i in range(self.hiddenLayer):
            for j in range(self.OutputLayer):
                self.WEIGHT_output[i][j] -= (self.learningRate * (DELTA_output[j] * self.OUTPUT_L1[i]))
                self.BIAS_output[j] -= (self.learningRate * DELTA_output[j])
      
        'Stage 3 - Error: HiddenLayer'
        delta_hidden = np.matmul(self.WEIGHT_output, DELTA_output)* self.deriv(self.OUTPUT_L1)
 
        'Stage 4 - Update weights HiddenLayer and InputLayer(x)'
        for i in range(self.OutputLayer):
            for j in range(self.hiddenLayer):
                self.WEIGHT_hidden[i][j] -= (self.learningRate * (delta_hidden[j] * x[i]))
                self.BIAS_hidden[j] -= (self.learningRate * delta_hidden[j])
                
    def show_err_graphic(self,v_erro,v_epoca):
        plt.figure(figsize=(9,4))
        plt.plot(v_epoca, v_erro, "m-",color="b", marker=11)
        plt.xlabel("Number of Epochs")
        plt.ylabel("Squared error (MSE) ");
        plt.title("Error Minimization")
        plt.show()

    def predict(self, X, y):
        'Returns the predictions for every element of X'
        my_predictions = []
        'Forward Propagation'
        forward = np.matmul(X,self.WEIGHT_hidden) + self.BIAS_hidden
        forward = np.matmul(forward, self.WEIGHT_output) + self.BIAS_output
                                 
        for i in forward:
            my_predictions.append(max(enumerate(i), key=lambda x:x[1])[0])
            
        array_score = []
        for i in range(len(my_predictions)):
            if my_predictions[i] == 0: 
                array_score.append([i, 'Iris-setosa', my_predictions[i], y[i]])
            elif my_predictions[i] == 1:
                 array_score.append([i, 'Iris-versicolour', my_predictions[i], y[i]])
            elif my_predictions[i] == 2:
                 array_score.append([i, 'Iris-virginica', my_predictions[i], y[i]])
                    
        dataframe = pd.DataFrame(array_score, columns=['_id', 'class', 'output', 'hoped_output'])
        return my_predictions, dataframe

    def fit(self, X, y):  
        count_epoch = 1
        total_error = 0
        n = len(X); 
        epoch_array = []
        error_array = []
        W0 = []
        W1 = []
        while(count_epoch <= self.max_epochs):
            for idx,inputs in enumerate(X): 
                self.output = np.zeros(self.classes_number)
                'Stage 1 - (Forward Propagation)'
                self.OUTPUT_L1 = self.activation((np.dot(inputs, self.WEIGHT_hidden) + self.BIAS_hidden.T))
                self.OUTPUT_L2 = self.activation((np.dot(self.OUTPUT_L1, self.WEIGHT_output) + self.BIAS_output.T))
                'Stage 2 - One-Hot-Encoding'
                if(y[idx] == 0): 
                    self.output = np.array([1,0,0]) #Class1 {1,0,0}
                elif(y[idx] == 1):
                    self.output = np.array([0,1,0]) #Class2 {0,1,0}

                
                square_error = 0
                for i in range(self.OutputLayer):
                    erro = (self.output[i] - self.OUTPUT_L2[i])**2
                    square_error = (square_error + (0.05 * erro))
                    total_error = total_error + square_error
         
                'Backpropagation : Update Weights'
                self.Backpropagation_Algorithm(inputs)
                
            total_error = (total_error / n)
            if((count_epoch % 50 == 0)or(count_epoch == 1)):
                print("Epoch ", count_epoch, "- Total Error: ",total_error)
                error_array.append(total_error)
                epoch_array.append(count_epoch)
                
            W0.append(self.WEIGHT_hidden)
            W1.append(self.WEIGHT_output)
             
                
            count_epoch += 1
        self.show_err_graphic(error_array,epoch_array)
        
        plt.plot(W0[0])
        plt.title('Weight Hidden update during training')
        plt.legend(['neuron1', 'neuron2', 'neuron3', 'neuron4', 'neuron5'])
        plt.ylabel('Value Weight')
        plt.show()
        
        plt.plot(W1[0])
        plt.title('Weight Output update during training')
        plt.legend(['neuron1', 'neuron2', 'neuron3'])
        plt.ylabel('Value Weight')
        plt.show()

        return self

In [65]:
dictionary = {'InputLayer':29, 'HiddenLayer':5, 'OutputLayer':1,
              'Epocas':700, 'LearningRate':0.005,'BiasHiddenValue':-1, 
              'BiasOutputValue':-1, 'ActivationFunction':'sigmoid'}

Perceptron = MultiLayerPerceptron(dictionary)
Perceptron.fit(X_train,y_train)

ValueError: ignored

In [83]:
import numpy as np


class Perceptron:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.activation_func = self._unit_step_func
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # init parameters
        self.weights = np.zeros(n_features)
        self.bias = 0

        y_ = np.array([1 if i > 0 else 0 for i in y])

        for _ in range(self.n_iters):

            for idx, x_i in enumerate(X):

                linear_output = np.dot(x_i, self.weights) + self.bias
                y_predicted = self.activation_func(linear_output)

                # Perceptron update rule
                update = self.lr * (y_[idx] - y_predicted)

                self.weights += update * x_i
                self.bias += update

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predicted = self.activation_func(linear_output)
        return y_predicted

    def _unit_step_func(self, x):
        return np.where(x >= 0, 1, 0)


# Testing
if __name__ == "__main__":
    # Imports
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split
    from sklearn import datasets

    def accuracy(y_true, y_pred):
        accuracy = np.sum(y_true == y_pred) / len(y_true)
        return accuracy

    X = df[colunas.drop('Class')]
    Y = df['Class']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

    p = Perceptron(learning_rate=0.01, n_iters=1000)
    p.fit(X_train, y_train)
    predictions = p.predict(X_test)

    print("Perceptron classification accuracy", accuracy(y_test, predictions))

UFuncTypeError: ignored

In [ ]:
def relu(z):
    return x * (x > 0)
def forward_propagation(X, parameters):
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}
    
    return A2, cache
    

In [ ]:
def backward_propagation(parameters, cache, X, Y):
    #number of training example
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
   
    dZ2 = A2-Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2,"db2": db2}
    
    return grads

In [ ]:
from scipy import stats

pred = np.array([])
k = 5
tam = len(distancias)
for i in range(tam):
  idxkneigh = distancias[i].argsort()[:k] # Pega os indices das k menores distancias
  vizinhos = [y_train[idxkneigh[j]] for j in range(k)] # Busca qual a classificação desses indices: red ou não
  mode_info = stats.mode(vizinhos) # Pega a moda das classificações dos KNN
  pred = np.append(pred, mode_info[0]) # Prediz

In [ ]:
acertos = 0
for i in range(tam):
  if pred[i] == y_test[i]:
    acertos += 1

print(acertos)

629


In [ ]:
acertos/(tam)

0.9828125

99% Precisão normalizado e com PCA

92% Precisão sem normalizar e sem PCA

98% Precisão normalizando e sem PCA